In [ ]:
#上課程式碼

# analyzing the Seoul bike data
# read in the data file
import pandas as pd
df = pd.read_csv('SeoulBikeData.csv', index_col=['Date'])
df = pd.read_csv('SeoulBikeData.csv', encoding='gbk', index_col=['Date'])
df

# select only the data entries of ['Functioning Day'] == 'Yes' and drop the 'Functioning Day' feature
data = df.copy()
print(data)
data = data[data['Functioning Day'] == 'Yes']
print(data)
data.pop('Functioning Day')
print(data)

# find the column names with odd codes and change the
print(data.columns)
data = data.rename(columns={'Temperature(癈)':'Temperature(C)', 'Dew point temperature(癈)':'Dew point temperature(C)'})
print(data.columns)

# convert the text features into numeric
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
print(data)
print(data['Seasons'])
data['Seasons'] = le.fit_transform(data['Seasons'])
data['Holiday'] = le.fit_transform(data['Holiday'])
print(data)

# extract the target variable
print(data)
target = data.pop('Rented Bike Count')
print(data)
print(target)

# start training and testing the multiple regression model
from sklearn.model_selection import train_test_split
data_train, data_test, target_train, target_test = train_test_split(data.values, target.values, test_size=0.2)

from sklearn.linear_model import LinearRegression
reg_model = LinearRegression()
reg_model.fit(data_train, target_train)
predictions = reg_model.predict(data_test)

print(reg_model.score(data_train, target_train).round(3))
print(reg_model.score(data_test, target_test).round(3))
